In [3]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
text = "Model-based RL don't need a value function for the policy"
print(tokenizer.tokenize(text))

['Model-based', 'RL', 'do', "n't", 'need', 'a', 'value', 'function', 'for', 'the', 'policy']


In [4]:
from nltk.tokenize import word_tokenize
print(word_tokenize(text))

['Model-based', 'RL', 'do', "n't", 'need', 'a', 'value', 'function', 'for', 'the', 'policy']


<font color=yellow>#대표적인 어간 추출(stemming) 기법인 Porter 및 Lancaster 추출 패키지를 불러오고 활용해봄 / Lemmatizer는 표제어 추출</font>

In [7]:
from nltk.stem import PorterStemmer, LancasterStemmer
stem1 = PorterStemmer()
stem2 = LancasterStemmer()
words = ["eat","ate","eaten","eating"]
print("Porter Stemmer       :", [stem1.stem(w) for w  in words])
print("Lancaster Stemmer    :", [stem2.stem(v) for v  in words])

Porter Stemmer       : ['eat', 'ate', 'eaten', 'eat']
Lancaster Stemmer    : ['eat', 'at', 'eat', 'eat']


In [13]:
from nltk import WordNetLemmatizer
lemm = WordNetLemmatizer()
words = ["eat","ate","eaten","eating"]
print("WordNet Lemmatizer:",[lemm.lemmatize(v,pos="v") for v in words])

WordNet Lemmatizer: ['eat', 'eat', 'eat', 'eat']


<font color=yellow>#불용어 제거</font>

In [14]:
from nltk.corpus import stopwords
print(stopwords.words('english')[:5])

['i', 'me', 'my', 'myself', 'we']


In [16]:
input_sentence = "We should all study hard for the exam."
stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(input_sentence)
result = []
for w in word_tokens:
    if w not in stop_words:
        result.append(w)
print(word_tokens)
print(result)

['We', 'should', 'all', 'study', 'hard', 'for', 'the', 'exam', '.']
['We', 'study', 'hard', 'exam', '.']
